In [2]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers, initializers, losses
from keras.wrappers.scikit_learn import KerasClassifier
import data_helper as dh
import numpy as np
from matplotlib import pyplot as plt


# parameter 
s_n = 53 # station number
v_len = 358 # one-hot encoding length + delay time
time_step_n = 10
node_n1 = 50 # hidden layer node
node_n2 = 100 # hidden layer node
epoch = 2
data_n = 20000 # 'all' or integer
train_data_n = 18000 # 학습 data 수


# RNN model
def RNN_model():
    model = Sequential()
    # model.add(Dense(node_n1, kernel_initializer=initializers.he_normal()))
    model.add(SimpleRNN(node_n2, batch_input_shape = (1, time_step_n, v_len), stateful=True))
    model.add(Dense(1, kernel_initializer=initializers.he_normal()))



    adam = optimizers.Adam(lr = 0.0001) # optimizer
    model.compile(loss = 'mean_squared_error', optimizer = adam, metrics = ['mean_squared_error'])
    
    return model


# data split
x, y = dh.readDataV2_nopad('./gong2_data_nopad.csv', count=data_n)
x_test = x[train_data_n:]
y_test = y[train_data_n:]
x = x[:train_data_n]
y = y[:train_data_n]

In [ ]:
# training
model = RNN_model()
print('======================= Training ================================')
for m in range(epoch):
    print('======================= epoch %d ================================' % (m+1))
    for l in range(train_data_n): # 열차마다 반복
        x1 = x[l] # 한 열차의 data
        x2 = [] # input으로 들어갈 set들
        for i in range(len(x1) - time_step_n):
            xxx = [] # 한 set
            for k in range(time_step_n):
                xxx.append(x1[i+k])
            x2.append(xxx) 
        x2 = np.array(x2)

        y1 = y[l] # 한 열차의 실제 지연시간 
        y2 = y1[time_step_n:] # set에 따른 정답
        y2 = np.array(y2)

        # print('train %d' % (l+1))
        model.fit(x2, y2, batch_size = 1, epochs = 1, verbose=0, shuffle=False)
        model.reset_states()
print('=================================================================')
model.save('./20000_100_nopad')

In [ ]:
# test
from keras.models import load_model
import pandas as pd

model = load_model('./20000_100_nopad')

total_accuracy = [] # accuracy 모음

y_pred = [] # 전체 예측값
for i in range(data_n - train_data_n): # 열차마다 반복
    print('======================= Train %d ================================' % (i+1))
    x_input_one = np.reshape(x_test[i][0:10], (1, time_step_n, v_len)) # 모델에 넣을 input, size: (1, time_step_n, v_len)
    #print(x_input_one[0,0:10,357])
    # if(x_input_one[0,9,357]==0):
    #     continue
    y_pred_one = np.array(y_test[i][0:10]) # 한 열차의 예측값들, 나중에 append를 해서 값 추가, 처음 time_step_n개의 data는 실제값, 처음 size: (time_step_n, 1)
    y_test_one = np.array(y_test[i]) # 한 열차의 실제 지연 시간, size: (53, 1)
    s_n = len(x_test[i])
    for k in range(s_n - time_step_n): # 한 열차에서 지연시간 예측
        y_step = model.predict(x_input_one) # 한 다음 지연 시간 예측
        y_pred_one = np.append(y_pred_one ,y_step, axis=0)

        x_input_one = np.append(x_input_one, np.reshape(x_test[i][time_step_n + k], (1, 1, v_len)), axis=1) # 모델에 들어갈 input에 feature vector 추가
        x_input_one[-1, -1, -1] = y_step # feature vector의 마지막 값은 예측 지연시간으로 대입
        x_input_one = np.delete(x_input_one, 0, axis=1) # 맨앞의 feature vector 삭제

    mse = losses.MeanSquaredError()
    print('loss: ', mse(y_pred_one[5:, :], y_test_one[5:, :]).numpy()) # 앞의 5개의 실제 지연시간을 제외한 값들의 loss

    # accuracy
    y_error = abs(y_pred_one[5:, :] - y_test_one[5:, :]) # 오차 계산
    r, __ = np.where(y_error < 60) # 일정 시간보다 작은 곳의 index
    acc_percent = len(r) / y_error.size * 100
    total_accuracy.append(acc_percent)
    print('accuracy: %.2f' %(acc_percent), '%')
    
    y_pred.append(y_pred_one) # 한 열차의 예측값 저장, size: ( 열차수, s_n, 1(지연 시간) )
    model.reset_states() # 모델 state 초기화

print('=================================================================')

print('Final accuracy: %.2f' %(sum(total_accuracy)/len(total_accuracy))) # 전체 성능

In [6]:
# save result

AD = []

for item in y_pred:
    line =[]
    for it in item:
        line.append(it[0])
    AD.append(line)
df = pd.DataFrame(AD)
df.to_csv('result_2000_100_1.csv')

# df = pd.DataFrame(list(y_pred))
# df.to_csv('result_2000_100.csv')

# save test data
AD = []

for item in y_test:
    line =[]
    for it in item:
        line.append(it[0])
    AD.append(line)
df = pd.DataFrame(AD)
df.to_csv('test_2000_100_1.csv')

# df = pd.DataFrame(list(y_test))
# df.to_csv('test_data_2000_100.csv')

In [ ]:
for i in range(20):
    plt.plot(y_pred[i], label='Predicted')
    plt.plot(y_test[i], label = 'Real')
    #plt.xlabel('Time of day')
    plt.ylabel('Delay Time')
    plt.legend()
    plt.show()